### 生成器到协成

### 1.yield 版本

In [6]:
import time


def one_task():
    print("begin task")
    print("     begin big_step")
    big_result = big_step()
    print(f"    end big_step {big_result}")
    print("end task")


def big_step():
    print(f"        begin small_step")
    small_coro = small_step()  # 接收返回的生成器
    while True:
        try:
            x = small_coro.send(None)
        except StopIteration as e:
            small_result = e.value
            break
        else:
            pass

    print(f"        end small_step with {small_result}")
    return small_result * 1000


def small_step():
    print("         休息一下")
    yield time.sleep(2)
    print("         努力工作")
    return 123


one_task()


begin task
     begin big_step
        begin small_step
         休息一下
         努力工作
        end small_step with 123
    end big_step 123000
end task


### 2.yield from 版本

In [4]:
import time


def one_task():
    print("begin task")
    print("     begin big_step")
    big_result = yield from big_step()
    print(f"    end big_step {big_result}")
    print("end task")


def big_step():
    print(f"        begin small_step")
    small_result = yield from small_step()
    print(f"        end small_step with {small_result}")
    return small_result * 1000


def small_step():
    print("         休息一下")
    yield from YieldFromObj((time.sleep, 2))
    print("         努力工作")
    return 123

class YieldFromObj:
    def __init__(self, value):
        self.value = value

    def __iter__(self):
        yield self

class Task:

    def __init__(self, coro):
        self.coro = coro

    def run(self):
        while True:
            try:
                x = self.coro.send(None)
            except StopIteration as e:
                result = e
                return
            else:
                # result = ?
                print("x是多少", x)
                resul = "不知道"
        print("-------", result)


t = Task(one_task())
t.run()

begin task
     begin big_step
        begin small_step
         休息一下
x是多少 <__main__.YieldFromObj object at 0x103d3ad90>
         努力工作
        end small_step with 123
    end big_step 123000
end task


In [10]:
import time


def one_task():
    print("begin task")
    print("     begin big_step")
    big_result = yield from big_step()
    print(f"    end big_step {big_result}")
    print("end task")


def big_step():
    print(f"        begin small_step")
    small_result = yield from small_step()
    print(f"        end small_step with {small_result}")
    return small_result * 1000


def small_step():
    print("         休息一下")
    yield from YieldFromObj((time.sleep, 2))
    print("         努力工作")
    return 123

class YieldFromObj:
    def __init__(self, value):
        self.value = value

    def __iter__(self):
        yield self

class Task:

    def __init__(self, coro):
        self.coro = coro

    def run(self):
        while True:
            try:
                x = self.coro.send(None)
            except StopIteration as e:
                result = e
                return
            else:
                print("         task----")
                func, val = x.value
                result = func(val)
        print("-------", result)


t = Task(one_task())
t.run()

begin task
     begin big_step
        begin small_step
         休息一下
         task----
         努力工作
        end small_step with 123
    end big_step 123000
end task


### 手动控制Task的运行

In [12]:
import time


def one_task():
    print("begin task")
    print("     begin big_step")
    big_result = yield from big_step()
    print(f"    end big_step {big_result}")
    print("end task")


def big_step():
    print(f"        begin small_step")
    small_result = yield from small_step()
    print(f"        end small_step with {small_result}")
    return small_result * 1000


def small_step():
    print("         休息一下")
    yield from YieldFromObj((time.sleep, 2))
    print("         努力工作")
    return 123

class YieldFromObj:
    def __init__(self, value):
        self.value = value

    def __iter__(self):
        yield self

class Task:

    def __init__(self, coro):
        self.coro = coro
        self._done = False
        self._result = None

    def run(self):
        if not self._done:
            try:
                x = self.coro.send(None)
            except StopIteration as e:
                self._result = e
                self._done = True
            else:
                pass
        else:
            print("task is done")


t = Task(one_task())
t.run()
# 等待两秒
for _ in range(10):
    print("做点其他事情")
    time.sleep(0.2)
t.run()
t.run()

begin task
     begin big_step
        begin small_step
         休息一下
做点其他事情
做点其他事情
做点其他事情
做点其他事情
做点其他事情
做点其他事情
做点其他事情
做点其他事情
做点其他事情
做点其他事情
         努力工作
        end small_step with 123
    end big_step 123000
end task
task is done


### EventLoop自动控制

In [38]:
import collections
import heapq
import itertools
import random
import time

class EventLoop:

    def __init__(self):
        self._ready = collections.deque()
        self._scheduled = []
        self._stopping = False

    def call_soon(self, callback, *args):
        self._ready.append((callback, args))

    def call_later(self, delay, callback, *args):
        heapq.heappush(self._scheduled, (time.time() + delay, callback, args))

    def run_forever(self):
        while True:
            self.run_one()
            if self._stopping:
                break

    def run_one(self):
        now = time.time()
        if self._scheduled:
            if self._scheduled[0][0] < now:
                _, cb, args = heapq.heappop(self._scheduled)
                self._ready.append((cb, args))

        num = len(self._ready)
        for i in range(num):
            cb, args = self._ready.popleft()
            cb(*args)

    def stop(self):
        self._stopping = True
        
    

In [39]:
loop = EventLoop()

def one_task():
    print("begin task")
    print("     begin big_step")
    big_result = yield from big_step()
    print(f"    end big_step {big_result}")
    print("end task")


def big_step():
    print(f"        begin small_step")
    small_result = yield from small_step()
    print(f"        end small_step with {small_result}")
    return small_result * 1000


def small_step():
    print("         休息一下")
    yield from YieldFromObj(2)
    print("         努力工作")
    return 123

class YieldFromObj:
    def __init__(self, value):
        self.value = value

    def __iter__(self):
        yield self

task_id_counter = itertools.count(1)

class Task:

    def __init__(self, coro):
        self.coro = coro
        self._done = False
        self._result = None
        self._id = f"Task-{next(task_id_counter)}"

    def run(self):
        print(f"-----{self._id}------")
        if not self._done:
            try:
                x = self.coro.send(None)
            except StopIteration as e:
                self._result = e.value
                self._done = True
            else:
                loop.call_later(x.value, self.run)
        else:
            print("task is done")
        print(f"-----{self._id}------")


for i in range(2):
    t = Task(one_task())
    loop.call_soon(t.run)
loop.call_later(5, loop.stop)
loop.run_forever()


-----Task-1------
begin task
     begin big_step
        begin small_step
         休息一下
-----Task-1------
-----Task-2------
begin task
     begin big_step
        begin small_step
         休息一下
-----Task-2------
-----Task-1------
         努力工作
        end small_step with 123
    end big_step 123000
end task
-----Task-1------
-----Task-2------
         努力工作
        end small_step with 123
    end big_step 123000
end task
-----Task-2------


In [43]:
import threading
loop = EventLoop()

def one_task():
    print("begin task")
    print("     begin big_step")
    big_result = yield from big_step()
    print(f"    end big_step {big_result}")
    print("end task")


def big_step():
    print(f"        begin small_step")
    small_result = yield from small_step()
    print(f"        end small_step with {small_result}")
    return small_result * 1000


def small_step():
    fut = Future()
    fake_io_read(fut)
    return fut


def fake_io_read(future):
    def read():
        # 耗时
        sleep_time = random.random()
        time.sleep(sleep_time)
        print("休息 %s" % sleep_time)
        future.set_result(random.randint(1, 100))

    threading.Thread(target=read).start()

class Future:
    def __init__(self):
        self._result = None
        self._done = False
        self._callbacks = []

    def set_result(self, result):
        if self._done:
            raise RuntimeError('future already done.')
        self._result = result
        self._done = True
        for cb in self._callbacks:
            loop.call_soon(cb)

    def result(self):
        if self._done:
            return self._result
        else:
            raise RuntimeError("future is not done.")

    def add_done_callback(self, callback):
        self._callbacks.append(callback)

    def __iter__(self):
        yield self
        return self.result()

task_id_counter = itertools.count(1)

class Task:

    def __init__(self, coro):
        self.coro = coro
        self._done = False
        self._result = None
        self._id = f"Task-{next(task_id_counter)}"

    def run(self):
        print(f"-----{self._id}------")
        if not self._done:
            try:
                x = self.coro.send(None)
            except StopIteration as e:
                self._result = e.value
                self._done = True
            else:
                x.add_done_callback(self.run)
        else:
            print("task is done")
        print(f"-----{self._id}------")


for i in range(2):
    t = Task(one_task())
    loop.call_soon(t.run)
loop.call_later(5, loop.stop)
loop.run_forever()


-----Task-1------
begin task
     begin big_step
        begin small_step
-----Task-1------
-----Task-2------
begin task
     begin big_step
        begin small_step
-----Task-2------
休息 0.5544231963646922
-----Task-2------
        end small_step with 64
    end big_step 64000
end task
-----Task-2------
休息 0.7289755517807259
-----Task-1------
        end small_step with 72
    end big_step 72000
end task
-----Task-1------
